# Metaclasses

Todas as classes são instâncias de `type`.

In [1]:
type('Olá, mundo!')

str

In [2]:
type(type('Olá, mundo!'))

type

In [3]:
type(type(type('Olá, mundo!')))

type

In [4]:
issubclass(type, object), issubclass(object, type)

(True, False)

In [5]:
isinstance(type, object), isinstance(object, type)

(True, True)

# Aplicando uma metaclasse

In [6]:
import evalsupport

class Bla(metaclass=evalsupport.MetaAleph):
    print("Corpo da classe Bla")
    def __init__(self):
        print("Corpo do Bla.__init__")

<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end
Corpo da classe Bla
<[500]> MetaAleph.__init__


In [7]:
Bla()

Corpo do Bla.__init__


## Personalizando descritores com metaclasses

Vamos usar o `LineItem` e os modelos já dados. Queremos reescrever o exemplo do decorator, mas usando herança.

In [8]:
import sys, pathlib
try:
    here = pathlib.Path(__file__)
except NameError:
    here = pathlib.Path(__vsc_ipynb_file__)
finally:
    here = here.parent.parent / '20_descritores_atributos'
    sys.path.append(here.__str__())
import models

In [9]:
class EntityMeta(type):
    def __init__(self, name, bases, attr_dict):
        super().__init__(name, bases, attr_dict)
        for key, attr in attr_dict.items():
            if isinstance(attr, models.Validated):
                type_name = type(attr).__name__
                attr.storage_name = f"_{type_name}#{key}"

class Entity(metaclass=EntityMeta):
    pass

In [10]:
class LineItem(Entity):
    descricao = models.NonBlank()
    peso = models.Quantity()
    preco = models.Quantity()

    def __init__(self, descricao, peso, preco):
        self.descricao = descricao
        self.peso = peso
        self.preco = preco
    def subtotal(self):
        return self.peso * self.preco

In [11]:
feijão = LineItem('Feijão Ble', 1.25, 10)
feijão.subtotal()

12.5

## Usando o novo `__prepare__`

É um método chamado antes do `__new__` da metaclasse e seu papel é criar um mapeamento dos atributos da classe.

In [20]:
import collections

class EntityMeta(type):
    @classmethod
    def __prepare__(cls, name, bases):
        print('-> Preparado!')
        x = collections.OrderedDict()
        x.update({
            "teste": "rastreamento"
        })
        return x
        
    def __init__(cls, name, bases, attr_dict):
        super().__init__(name, bases, attr_dict)
        cls._field_names = []
        for key, attr in attr_dict.items():
            if isinstance(attr, models.Validated):
                type_name = type(attr).__name__
                attr.storage_name = f"_{type_name}#{key}"
                cls._field_names.append(key)
            elif key == 'teste':
                cls._field_names.append(key)

class Entity(metaclass=EntityMeta):
    @classmethod
    def field_names(cls):
        for name in cls._field_names:
            yield name

class LineItem(Entity):
    descricao = models.NonBlank()
    peso = models.Quantity()
    preco = models.Quantity()

    def __init__(self, descricao, peso, preco):
        self.descricao = descricao
        self.peso = peso
        self.preco = preco
    def subtotal(self):
        return self.peso * self.preco

-> Preparado!
-> Preparado!


In [21]:
feijão = LineItem('Feijão Ble', 1.25, 10)
feijão.subtotal()

12.5

O método continua funcionando. O que fizemos foi preparar a estrutura de dados apropriada antes de instanciar a entidade.

Observe que o objeto retornado por `__prepare__` é capturado em `.attr_dict`, que depois é mapeado como atributo com o seu respectivo valor.

In [25]:
feijão.teste

'rastreamento'

## Classes como objetos

Vale a leitura dos atributos especiais da [própria documentação](https://docs.python.org/pt-br/3/library/stdtypes.html#special-attributes).